In [9]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import date
from pandas.tseries.offsets import BDay # Para calcular dias úteis
from sklearn.metrics import mean_squared_error, mean_absolute_error # Para adicionar métricas

# --- Configurações Iniciais ---
# LISTA DE AÇÕES A SEREM AVALIADAS
LISTA_DE_ACOES = ["VALE3.SA", "PETR4.SA", "ITUB4.SA"]
CAMINHO_DADOS_PROCESSADOS = "../data/03_processed"
CAMINHO_MODELOS = "../models"

# Parâmetros (Constantes Globais)
JANELA_DE_TEMPO = 60
PERCENTUAL_TREINO = 0.8

print("Ambiente configurado e constantes definidas.")


# --- FUNÇÃO DE PREVISÃO MULTI-PASSO (LOOP) ---
def prever_n_dias(model, X_ultima_sequencia, scaler, n_passos):
    """
    Realiza a previsão de n_passos (dias) usando a técnica de loop.
    """
    sequencia_atual = X_ultima_sequencia.copy()
    previsoes_normalizadas = []

    for _ in range(n_passos):
        X_input = np.expand_dims(sequencia_atual, axis=0)
        previsao_normalizada = model.predict(X_input, verbose=0)
        previsoes_normalizadas.append(previsao_normalizada[0, 0])

        # Atualiza a sequência de entrada para o próximo passo
        sequencia_atual = np.roll(sequencia_atual, -1, axis=0)
        sequencia_atual[-1] = previsao_normalizada[0, 0]

    previsoes_normalizadas = np.array(previsoes_normalizadas).reshape(-1, 1)
    previsoes_reais = scaler.inverse_transform(previsoes_normalizadas)

    return previsoes_reais

Ambiente configurado e constantes definidas.


In [10]:
# --- FUNÇÃO PRINCIPAL DE AVALIAÇÃO E PREVISÃO ---

def avaliar_e_prever_acao(ticker, janela_de_tempo, percentual_treino):
    print(f"\n=======================================================")
    print(f"========= INICIANDO AVALIAÇÃO: {ticker} =========")
    print(f"=======================================================")

    # --- 1. CARREGAMENTO E VERIFICAÇÃO ---
    try:
        # Carrega Modelo (usando .keras)
        caminho_modelo = os.path.join(CAMINHO_MODELOS, f"{ticker}_lstm_model.keras")
        model = tf.keras.models.load_model(caminho_modelo)

        # Carrega o scaler (usando .joblib)
        caminho_scaler = os.path.join(CAMINHO_DADOS_PROCESSADOS, f"{ticker}_scaler.joblib")
        scaler = joblib.load(caminho_scaler)

        # Carrega os dados de teste e Treino
        X_train = np.load(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{ticker}_X_train.npy'))
        X_test = np.load(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{ticker}_X_test.npy'))
        y_test = np.load(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{ticker}_y_test.npy'))

    except Exception as e:
        print(f"ERRO CRÍTICO: Falha ao carregar arquivos para {ticker}. Detalhe: {e}")
        return


    # --- 2. AVALIAÇÃO DO TESTE (SINGLE-STEP PREDICTION) ---
    print("\n[STEP 2] Avaliando a performance no conjunto de teste...")
    previsoes_normalizadas_teste = model.predict(X_test, verbose=0)
    previsoes_reais_teste = scaler.inverse_transform(previsoes_normalizadas_teste)
    y_test_reais = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Cálculo das Métricas
    rmse = np.sqrt(mean_squared_error(y_test_reais, previsoes_reais_teste))
    mae = mean_absolute_error(y_test_reais, previsoes_reais_teste)
    print(f"RMSE (Erro Quadrático Médio): {rmse:.4f}")
    print(f"MAE (Erro Absoluto Médio): {mae:.4f}")


    # --- 3. PREPARAÇÃO PARA PREVISÃO FUTURA ---

    # Recarregar o arquivo CSV original (para obter as datas)
    caminho_arquivo_original = os.path.join("../data/01_raw", f"{ticker}.csv")
    df_original = pd.read_csv(
        caminho_arquivo_original,
        header=None,
        skiprows=3,
        index_col=0,
        parse_dates=[0]
    )
    df_original.dropna(inplace=True)

    ultima_data_dados = df_original.index[-1].date()
    hoje = date.today()

    print(f"\n[STEP 3] Preparando a previsão futura...")
    print(f"Última data histórica nos dados: {ultima_data_dados}")

    # Calcular o número de passos (dias úteis) que queremos prever
    try:
        # Range de datas inicia no dia útil seguinte e vai até hoje
        datas_futuras = pd.date_range(start=ultima_data_dados + BDay(1), end=hoje, freq='B')
        n_dias_prever = len(datas_futuras)
        print(f"Dias úteis a prever até hoje ({hoje}): {n_dias_prever}")
    except ValueError:
        n_dias_prever = 0
        datas_futuras = pd.DatetimeIndex([])
        print("A data histórica é igual ou posterior a hoje. Sem previsão futura necessária.")


    # --- 4. EXECUÇÃO DA PREVISÃO MULTI-PASSO ---
    previsoes_futuras_reais = np.array([])

    if n_dias_prever > 0:
        # Usa a última sequência do X_test como ponto de partida
        X_ultima_sequencia = X_test[-1]
        previsoes_futuras_reais = prever_n_dias(model, X_ultima_sequencia, scaler, n_dias_prever)

        valor_previsto_hoje = previsoes_futuras_reais[-1][0]
        print(f"Previsão Multi-Passo Concluída. Valor previsto para hoje ({hoje}): R$ {valor_previsto_hoje:.2f}")


    # --- 5. PREPARAÇÃO DO GRÁFICO CONSOLIDADO ---

    # Combina datas e valores para o plot
    dados_fechamento_original = df_original.iloc[:, 1].values.reshape(-1, 1)
    datas_reais_com_janela = df_original.index[janela_de_tempo:].to_numpy() # Usa o argumento 'janela_de_tempo'

    # Datas de teste (incluindo o espaço para a janela)
    datas_para_plotar_teste = datas_reais_com_janela[len(X_train):]

    # Junta as datas de teste com as datas futuras
    datas_teste_e_futuras = np.concatenate((datas_para_plotar_teste, datas_futuras.to_numpy()))

    # Cria a linha do preço REAL (só vai até o final dos dados de teste)
    y_reais_plot = np.concatenate((y_test_reais.flatten(), [None] * len(previsoes_futuras_reais)))

    # Cria a linha da PREVISÃO (segue do teste para o futuro)
    y_previsao_plot = np.concatenate((previsoes_reais_teste.flatten(), previsoes_futuras_reais.flatten()))


    # --- 6. CRIAÇÃO DO GRÁFICO INTERATIVO ---

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=datas_teste_e_futuras,
        y=y_reais_plot,
        mode='lines+markers',
        name='Preço Real (Teste)',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=datas_teste_e_futuras,
        y=y_previsao_plot,
        mode='lines',
        name='Previsão do Modelo (Teste + Futuro)',
        line=dict(color='red', dash='dash')
    ))

    fig.update_layout(
        title=f'Preço Real vs. Previsão LSTM (Teste + Futuro) para {ticker}',
        xaxis_title='Data',
        yaxis_title='Preço de Fechamento (R$)',
        legend_title='Legenda',
        template='plotly_dark'
    )

    fig.show()

In [11]:
# --- EXECUÇÃO DO LOOP PRINCIPAL PARA TODAS AS AÇÕES ---

print("Iniciando o loop de avaliação e previsão...")

for acao in LISTA_DE_ACOES:
    avaliar_e_prever_acao(acao, JANELA_DE_TEMPO, PERCENTUAL_TREINO)

print("\n\n✅ Avaliação e previsão para todas as ações concluída.")

Iniciando o loop de avaliação e previsão...

========= INICIANDO AVALIAÇÃO: VALE3.SA =========

[STEP 2] Avaliando a performance no conjunto de teste...
RMSE (Erro Quadrático Médio): 2.5454
MAE (Erro Absoluto Médio): 1.4147

[STEP 3] Preparando a previsão futura...
Última data histórica nos dados: 2025-10-30
Dias úteis a prever até hoje (2025-11-07): 6
Previsão Multi-Passo Concluída. Valor previsto para hoje (2025-11-07): R$ 61.97



========= INICIANDO AVALIAÇÃO: PETR4.SA =========

[STEP 2] Avaliando a performance no conjunto de teste...
RMSE (Erro Quadrático Médio): 1.1679
MAE (Erro Absoluto Médio): 0.6389

[STEP 3] Preparando a previsão futura...
Última data histórica nos dados: 2025-10-30
Dias úteis a prever até hoje (2025-11-07): 6
Previsão Multi-Passo Concluída. Valor previsto para hoje (2025-11-07): R$ 27.60



========= INICIANDO AVALIAÇÃO: ITUB4.SA =========

[STEP 2] Avaliando a performance no conjunto de teste...
RMSE (Erro Quadrático Médio): 1.2699
MAE (Erro Absoluto Médio): 0.5131

[STEP 3] Preparando a previsão futura...
Última data histórica nos dados: 2025-10-30
Dias úteis a prever até hoje (2025-11-07): 6
Previsão Multi-Passo Concluída. Valor previsto para hoje (2025-11-07): R$ 37.83




✅ Avaliação e previsão para todas as ações concluída.
